<a href="https://colab.research.google.com/github/sebaherrera74/DASO-Tp1/blob/master/Datos_externos_archivos_locales%2C_Drive%2C_Hojas%C2%A0de%C2%A0c%C3%A1lculo_y_Cloud%C2%A0Storage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este notebook proporciona recetas para cargar y guardar datos de fuentes externas.

# Sistema de archivos local

## Subir archivos desde el sistema de archivos local

<code>files.upload</code> muestra un diccionario de los archivos que se cargaron.
El diccionario está codificado según el nombre de archivo y los valores son los datos que se subieron.

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

## Descargar archivos en el sistema de archivos local

<code>files.download</code> invocará una descarga del archivo desde el navegador a tu computadora local.


In [ ]:
from google.colab import files

with open('example.txt', 'w') as f:
  f.write('some content')

files.download('example.txt')

# Google Drive

Puedes acceder a archivos en Drive de varias maneras, incluidas las siguientes:
- Activación de Google Drive en la máquina virtual del entorno de ejecución
- Usar un wrapper alrededor de la API, como <a href="https://pythonhosted.org/PyDrive/">PyDrive</a>
- Usar la <a href="https://developers.google.com/drive/v3/web/about-sdk">API nativa de REST</a>



Puedes ver ejemplos de cada caso más abajo.

## Activar Google Drive de forma local

El ejemplo siguiente muestra cómo activar Google Drive en el entorno de ejecución mediante un código de autorización y cómo escribir y leer archivos allí. Una vez ejecutado, podrás ver el archivo nuevo &#40;<code>foo.txt</code>&#41; en <a href="https://drive.google.com/">https://drive.google.com/</a>.

Esta configuración solo permite leer, escribir y mover archivos. Para modificar de forma programática la configuración de uso compartido o algún otro metadato, prueba con las opciones de más abajo.

<strong>Nota:</strong> Si utilizas el botón "Activar unidad de Drive" en el navegador de archivos, no necesitas ningún código de autenticación para los notebooks que solo modificó el usuario actual.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
with open('/content/drive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/drive/My\ Drive/foo.txt

In [ ]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

## PyDrive

En los ejemplos que aparecen a continuación, se muestran la autenticación y la carga y descarga de archivos mediante PyDrive. Puedes encontrar más ejemplos en la <a href="https://pythonhosted.org/PyDrive/">documentación de PyDrive</a>.

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Autentica y crea el cliente de PyDrive.


In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Crea y sube un archivo de texto.


In [ ]:
uploaded = drive.CreateFile({'title': 'Sample upload.txt'})
uploaded.SetContentString('Sample upload file content')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Carga un archivo por su ID y luego imprime su contenido.


In [ ]:
downloaded = drive.CreateFile({'id': uploaded.get('id')})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

## API de REST de Drive

Para usar la API de Drive, primero debemos autenticar y construir un cliente de API.


In [ ]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

Con este cliente, podemos usar cualquiera de las funciones de la <a href="https://developers.google.com/drive/v3/reference/">referencia de la API de Google Drive</a>. Puedes ver algunos ejemplos debajo.


### Crear un archivo de Drive nuevo con datos de Python

Primero, crea un archivo local para poder subirlo después.

In [ ]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

Sube el archivo utilizando el método <a href="https://developers.google.com/drive/v3/reference/files/create"><code>files.create</code></a>. Puedes encontrar más detalles sobre cómo subir archivos en la <a href="https://developers.google.com/drive/v3/web/manage-uploads">documentación para desarrolladores</a>.

In [ ]:
from googleapiclient.http import MediaFileUpload

file_metadata = {
  'name': 'Sample file',
  'mimeType': 'text/plain'
}
media = MediaFileUpload('/tmp/to_upload.txt', 
                        mimetype='text/plain',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id')))

Después de ejecutar la celda anterior, se mostrará un archivo nuevo con el nombre "Sample file" en <a href="https://drive.google.com/">https://drive.google.com/</a>.

### Descargar datos de un archivo de Drive a Python

Descarga el archivo que subimos anteriormente.

In [ ]:
file_id = created.get('id')

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

downloaded.seek(0)
print('Downloaded file contents are: {}'.format(downloaded.read()))

Para descargar un archivo diferente, establece el <code>file&#95;id</code> anterior con el ID de ese archivo, que se verá así: "1uBtlaggVyWshwcyP6kEI-y&#95;W3P8D26sz".

# Hojas de cálculo de Google

Los ejemplos de abajo usan la biblioteca de código abierto <a href="https://github.com/burnash/gspread"><code>gspread</code></a> para interactuar con Hojas de cálculo de Google.

Importa la biblioteca, autentica y crea la interfaz para Hojas de cálculo.

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

A continuación, verás un pequeño conjunto de ejemplos de <code>gspread</code>. Puedes encontrar más ejemplos en la <a href="https://github.com/burnash/gspread#more-examples">página de GitHub de <code>gspread</code></a>.

## Crear una hoja nueva con datos de Python

In [ ]:
sh = gc.create('My cool spreadsheet')

Después de ejecutar la celda anterior, se mostrará una nueva hoja de cálculo con el nombre "My cool spreadsheet" en <a href="https://sheets.google.com/">https://sheets.google.com</a>.

Abre la hoja nueva y agrega datos aleatorios.

In [ ]:
worksheet = gc.open('My cool spreadsheet').sheet1

cell_list = worksheet.range('A1:C2')

import random
for cell in cell_list:
  cell.value = random.randint(1, 10)

worksheet.update_cells(cell_list)

## Descargar datos de una hoja de cálculo en Python como un DataFrame de Pandas

Vuelve a leer los datos aleatorios que insertamos anteriormente y convierte el resultado en un <a href="https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html">DataFrame de Pandas</a>.

In [ ]:
worksheet = gc.open('My cool spreadsheet').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

import pandas as pd
pd.DataFrame.from_records(rows)

# Google Cloud Storage &#40;GCS&#41;

Para usar Colaboratory con GCS, deberás crear un <a href="https://cloud.google.com/storage/docs/projects">proyecto de Google Cloud</a> o utilizar uno existente.

A continuación, especifica el ID del proyecto:

In [ ]:
project_id = 'Your_project_ID_here'

Los archivos en GCS están contenidos en <a href="https://cloud.google.com/storage/docs/key-terms#buckets">depósitos</a>.

Los depósitos deben tener un nombre global único, así que generamos uno aquí.

In [ ]:
import uuid
bucket_name = 'colab-sample-bucket-' + str(uuid.uuid1())

Para acceder a GCS, debemos realizar la autenticación.

In [ ]:
from google.colab import auth
auth.authenticate_user()

Se puede acceder a GCS a través de la línea de comandos <code>gsutil</code> o a través de la API nativa de Python.

## `gsutil`

Primero, configuramos <code>gsutil</code> para usar el proyecto que ya especificamos anteriormente mediante <code>gcloud</code>.

In [ ]:
!gcloud config set project {project_id}

Crea un archivo local para poder subirlo después.

In [ ]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

Crea un bucket donde subir el archivo &#40;<a href="https://cloud.google.com/storage/docs/gsutil/commands/mb">documentación</a>&#41;.

In [ ]:
!gsutil mb gs://{bucket_name}

Copiar el archivo en nuestro nuevo bucket &#40;<a href="https://cloud.google.com/storage/docs/gsutil/commands/cp">documentación</a>&#41;.

In [ ]:
!gsutil cp /tmp/to_upload.txt gs://{bucket_name}/

Vuelca el contenido de nuestro archivo recién copiado para asegurarte de que todo funcionó &#40;<a href="https://cloud.google.com/storage/docs/gsutil/commands/cat">documentación</a>&#41;.


In [ ]:
!gsutil cat gs://{bucket_name}/to_upload.txt

In [ ]:
#@markdown Una vez que se suba el archivo, aparecerán los datos en el navegador de almacenamiento de Cloud Console para el proyecto:
print('https://console.cloud.google.com/storage/browser?project=' + project_id)

Finalmente, descargaremos el archivo que acabamos de subir en el ejemplo anterior. Solo tienes que invertir el orden en el comando <code>gsutil cp</code>.

In [ ]:
!gsutil cp gs://{bucket_name}/to_upload.txt /tmp/gsutil_download.txt
  
# Imprime el resultado para asegurarte de que la transferencia funcionó.
!cat /tmp/gsutil_download.txt

## API de Python

Estos fragmentos se basan en <a href="https://github.com/GoogleCloudPlatform/storage-file-transfer-json-python/blob/master/chunked_transfer.py">un ejemplo más grande</a> que muestra usos adicionales de la API.

Primero, creamos el cliente de servicio.

In [ ]:
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')

Crea un archivo local para poder subirlo después.

In [ ]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

Crea un bucket en el proyecto especificado más arriba.

In [ ]:
# Usa un nombre de bucket diferente, único en todo el mundo, a partir del ejemplo anterior de gsutil.
import uuid
bucket_name = 'colab-sample-bucket-' + str(uuid.uuid1())

body = {
  'name': bucket_name,
  # For a full list of locations, see:
  # https://cloud.google.com/storage/docs/bucket-locations
  'location': 'us',
}
gcs_service.buckets().insert(project=project_id, body=body).execute()
print('Done')

Sube el archivo a nuestro bucket recién creado.

In [ ]:
from googleapiclient.http import MediaFileUpload

media = MediaFileUpload('/tmp/to_upload.txt', 
                        mimetype='text/plain',
                        resumable=True)

request = gcs_service.objects().insert(bucket=bucket_name, 
                                       name='to_upload.txt',
                                       media_body=media)

response = None
while response is None:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, response = request.next_chunk()

print('Upload complete')

In [ ]:
#@markdown Una vez que se suba el archivo, aparecerán los datos en el navegador de almacenamiento de Cloud Console para el proyecto:
print('https://console.cloud.google.com/storage/browser?project=' + project_id)

Descarga el archivo que acabamos de subir.

In [ ]:
from apiclient.http import MediaIoBaseDownload

with open('/tmp/downloaded_from_gcs.txt', 'wb') as f:
  request = gcs_service.objects().get_media(bucket=bucket_name,
                                            object='to_upload.txt')
  media = MediaIoBaseDownload(f, request)

  done = False
  while not done:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, done = media.next_chunk()

print('Download complete')

Inspecciona el archivo descargado.


In [ ]:
!cat /tmp/downloaded_from_gcs.txt